In [46]:
import pandas as pd
import numpy as np
import math

In [47]:
en1 = pd.read_csv('EncounterData_cleaned.csv')
#noticed a problem with commas in the csv, fixed that first...
#now the fields are messy, fixed fields
#now sometimes they add extra description for physicians, adding new field
#now this messes up with no data for rows with no extra description... going to move everything with a number in that field 
#over

C:\Users\ahapp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [48]:
en1_df = pd.DataFrame(en1)

In [69]:
en1_df.iloc[1][6]

'29116 '

In [70]:
en1_df

,enc_number,enc_timestamp,enc_billable_ind,enc_orig_site_id,enc_qualifying_ind,enc_service_line_unscrubbed,en_type_prac,enc_pat_drvs_number,client_service_line_id,service_line_mapped
0,3544519,2015-10-20 19:12:32.000,N,2.0,Y,Case Management,168775,325.0,Behavioral Health,NaN
1,3544709,2015-10-20 21:49:43.000,N,1.0,N,MD_Family Practitioner,29116,63.0,Primary Care,NaN
2,3545122,1900-10-19 22:56:11.000,N,1.0,N,LPC_Mental Health Practitioner,51591,62.0,Behavioral Health,NaN
3,3545157,2015-10-20 20:54:41.000,N,2.0,N,Other,135474,330.0,Specialty,NaN
4,3545517,2015-10-19 10:31:33.000,N,1.0,N,UDS_Education Specialists,59088,73.0,Behavioral Health,NaN
...,...,...,...,...,...,...,...,...,...,...
75582,31438998,2019-05-24 17:38:40.000,N,16.0,N,NULL,1970350,-1.0,Ignore,NaN
75583,31439300,2019-05-24 07:46:23.000,N,16.0,N,MD,Family Practitioner,1951883.0,304,Primary Care
75584,31439474,2019-05-24 09:15:00.000,N,16.0,Y,NP,Nurse Practitioner,2121235.0,90,Primary Care
75585,31441936,2019-05-26 11:51:56.000,N,3.0,N,Nurse Practitioner,Family Practice,238983.0,143,Primary Care


In [61]:
en_type_prac = en1_df.drop(en1_df.columns[[0,1,2,3,4,5,7,8,9]], axis =1)

In [ ]:
#going to separate into two piles: one is extra, other is number

In [90]:
type(en_type_prac.iloc[1][0])

IndexError: index 6 is out of bounds for axis 0 with size 1

In [132]:
en1_df.iloc[75586][6]

nan

In [115]:
en_later_new = []

for i in range(len(en1_df)):
    spot = (str(en1_df.iloc[i][6]).lower())
    if spot.isupper() or spot.islower() == True:
        en_later_new.append([en1_df.iloc[i][6],en1_df.iloc[i][7],en1_df.iloc[i][8],en1_df.iloc[i][9]])
    else:
        en_later_new.append(['None Listed',en1_df.iloc[i][6],en1_df.iloc[i][7],en1_df.iloc[i][8]])
#had some problems with spaces being interpreted as letters :C I found a fix though....


In [117]:
en_later_new = pd.DataFrame(en_later_new)

In [118]:
en_later_new

,0,1,2,3
0,None Listed,168775,325.0,Behavioral Health
1,None Listed,29116,63.0,Primary Care
2,None Listed,51591,62.0,Behavioral Health
3,None Listed,135474,330.0,Specialty
4,None Listed,59088,73.0,Behavioral Health
...,...,...,...,...
75582,None Listed,1970350,-1.0,Ignore
75583,Family Practitioner,1951883.0,304,Primary Care
75584,Nurse Practitioner,2121235.0,90,Primary Care
75585,Family Practice,238983.0,143,Primary Care


In [79]:
pd.DataFrame(en_later_new)

,0,1,2,3
0,168775,325.0,Behavioral Health,NaN
1,29116,63.0,Primary Care,NaN
2,51591,62.0,Behavioral Health,NaN
3,135474,330.0,Specialty,NaN
4,59088,73.0,Behavioral Health,NaN
...,...,...,...,...
75582,1970350,-1.0,Ignore,NaN
75583,Family Practitioner,1951883.0,304,Primary Care
75584,Nurse Practitioner,2121235.0,90,Primary Care
75585,Family Practice,238983.0,143,Primary Care


In [ ]:
#now to bring them together and sing rejoice

In [139]:
en1_df_scratched = en1_df.drop(en1_df.columns[[6,7,8,9]], axis = 1)
en1_df_scratched['en_type_prac'] = en_later_new[0]
en1_df_scratched['enc_pat_drvs_number'] = en_later_new[1]
en1_df_scratched['client_service_line_id'] = en_later_new[2]
en1_df_scratched['service_line_mapped'] = en_later_new[3]

In [176]:

df_final = en1_df_scratched
enc_pat_drvs_number = []
for i in range(len(df_final['enc_pat_drvs_number'])):
                  
            if df_final.iloc[i]['enc_pat_drvs_number'] == '  ':
                       enc_pat_drvs_number.append(0)
            else:
                       enc_pat_drvs_number.append(df_final.iloc[i]['enc_pat_drvs_number'])
            


In [178]:
df_final['enc_pat_drvs_number'] = enc_pat_drvs_number

In [179]:
df_final

,enc_number,enc_timestamp,enc_billable_ind,enc_orig_site_id,enc_qualifying_ind,enc_service_line_unscrubbed,en_type_prac,enc_pat_drvs_number,client_service_line_id,service_line_mapped
0,3544519,2015-10-20 19:12:32.000,N,2.0,Y,Case Management,None Listed,168775,325.0,Behavioral Health
1,3544709,2015-10-20 21:49:43.000,N,1.0,N,MD_Family Practitioner,None Listed,29116,63.0,Primary Care
2,3545122,1900-10-19 22:56:11.000,N,1.0,N,LPC_Mental Health Practitioner,None Listed,51591,62.0,Behavioral Health
3,3545157,2015-10-20 20:54:41.000,N,2.0,N,Other,None Listed,135474,330.0,Specialty
4,3545517,2015-10-19 10:31:33.000,N,1.0,N,UDS_Education Specialists,None Listed,59088,73.0,Behavioral Health
...,...,...,...,...,...,...,...,...,...,...
75582,31438998,2019-05-24 17:38:40.000,N,16.0,N,NULL,None Listed,1970350,-1.0,Ignore
75583,31439300,2019-05-24 07:46:23.000,N,16.0,N,MD,Family Practitioner,1951883.0,304,Primary Care
75584,31439474,2019-05-24 09:15:00.000,N,16.0,Y,NP,Nurse Practitioner,2121235.0,90,Primary Care
75585,31441936,2019-05-26 11:51:56.000,N,3.0,N,Nurse Practitioner,Family Practice,238983.0,143,Primary Care


In [180]:
df_final.to_csv('Encounter_cleaned_df.csv', index=False)#done with encounter file

In [143]:
#next is persons csv, did similar stuff as i did to the first one already

In [165]:
persons = pd.read_csv('PersonData.csv')

In [166]:
persons = persons.fillna('0')

In [182]:
persons['pat_drvs_number'] = persons['pat_drvs_number'].astype(int)

In [183]:
persons.to_csv('personcleaned.csv', index=False)#make sure index=False is there.... hehehehe

In [147]:
persons['pat_homelessness_status'].unique()

array(['Not Homeless', nan, 'Shelter', 'Street', 'Transitional',
       'Unknown/Unreported', 'Doubling Up', 'Unknown', 'Homeless Shelter',
       'Other', 'NOT HOMELESS', 'Transitional Housing', 'No',
       'CCH Housing/Voucher', 'Unmapped', 'Homeless', 'HOMELESS SHELTER',
       'S', 'T', 'N', 'H', 'Independent Housing', 'Unknown/Unspecified',
       'DOUBLING UP', 'PUBLIC HOUSING', 'Yes', 'UNKNOWN', '*N/A', 'D'],
      dtype=object)

In [148]:
#after looking, all looked fine... never know though o.0

In [149]:
attr = pd.read_csv('AttributionData.csv')

In [159]:
attr.to_csv('AttributionData_cleaned.csv', index=False)

In [ ]:
#again, after inspection, all look good to me!
